<a href="https://colab.research.google.com/github/Baeltor/PCD/blob/prog/NotebooksMLII/Redes_neuronales_detecci%C3%B3n_de_fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import zipfile
import csv

### Importación de datos

* Cómo dataframe

In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/mlg-ulb/creditcardfraud/versions/3


In [15]:
os.listdir(path)

['creditcard.csv']

In [16]:
df = pd.read_csv(path + '/creditcard.csv')

In [17]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [19]:
df['Class'].value_counts(normalize=True)

,proportion
Class,
0,0.998273
1,0.001727


## Datos cómo arreglos

* Los volvemos arreglos de numpy para no generar conflictos con los datos de ingreso en keras

In [20]:
# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = path + '/creditcard.csv'

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


### Dividiendo el conjunto de datos en entrenamiento, pruebas y validación

In [21]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


In [22]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


### transfromación de los datos (escalado estándar) por medio de las medidas estadístricas asociadas a los datos

In [23]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

## Definición de la red neuronal secualcial, asociada al conjunto de datos

In [25]:
import keras

model = keras.Sequential(
    [
        keras.Input(shape=train_features.shape[1:], name='Entrada'),
        keras.layers.Dense(256, activation="relu", name='Oculta_1'),
        keras.layers.Dense(256, activation="relu", name='Oculta_2'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu", name='Oculta_3'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid", name='Salida'),
    ]
)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Oculta_1 (Dense)                     │ (None, 256)                 │           7,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Oculta_2 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Oculta_3 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Salida (Dense)                       │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 139,777 (546.00 KB)

 Trainable params: 139,777 (546.00 KB)

 Non-trainable params: 0 (0.00 B)

### Cálculo de métricas con respecto a la proyección del modelo calculado

In [26]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.keras")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 12s - 104ms/step - fn: 43.0000 - fp: 23930.0000 - loss: 2.2765e-06 - precision: 0.0154 - recall: 0.8969 - tn: 203499.0000 - tp: 374.0000 - val_fn: 4.0000 - val_fp: 6041.0000 - val_loss: 0.3316 - val_precision: 0.0116 - val_recall: 0.9467 - val_tn: 50845.0000 - val_tp: 71.0000
Epoch 2/30
112/112 - 7s - 63ms/step - fn: 31.0000 - fp: 7603.0000 - loss: 1.3006e-06 - precision: 0.0483 - recall: 0.9257 - tn: 219826.0000 - tp: 386.0000 - val_fn: 10.0000 - val_fp: 1069.0000 - val_loss: 0.0622 - val_precision: 0.0573 - val_recall: 0.8667 - val_tn: 55817.0000 - val_tp: 65.0000
Epoch 3/30
112/112 - 9s - 81ms/step - fn: 27.0000 - fp: 8232.0000 - loss: 1.2829e-06 - precision: 0.0452 - recall: 0.9353 - tn: 219197.0000 - tp: 390.0000 - val_fn: 7.0000 - val_fp: 781.0000 - val_loss: 0.0700 - val_precision: 0.0801 - val_recall: 0.9067 - val_tn: 56105.0000 - val_tp: 68.0000
Epoch 4/30
112/112 - 7s - 61ms/step - fn: 21.0000 - fp: 6126.0000 - loss: 9.4452e-07 - precision: 0.0607 - recal

In [28]:
model.evaluate(val_features, val_targets, verbose=2)

1781/1781 - 4s - 2ms/step - fn: 6.0000 - fp: 1146.0000 - loss: 0.0787 - precision: 0.0568 - recall: 0.9200 - tn: 55740.0000 - tp: 69.0000


[0.07865195721387863,
 6.0,
 1146.0,
 55740.0,
 69.0,
 0.056790124624967575,
 0.9200000166893005]